In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.0 MB/s eta 0:00:00
     ━━━━

In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["HF_COLAB"] = userdata.get('HF_COLAB')

In [4]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_COLAB'])
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ['HF_COLAB'])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
text = """
Tab IDA 625mg (21)
1+1+1
Tab lora 10mg (10)
Tab lora 10mg
0+0+1
Cap- Ometid 20mg
1+0+1
8+0+1 2mg (10)
Tab Brizy 2mg(10)
Tab Brizy 2mg
1+0+1
Tab Napa 500mg
2+2+2
Rhinozol nasal
drops 0.1% 2-3
drops in each
nostril 4 times daily

This is a medical raw text comes from OCR. This contains Medicine Type (which is either Tab or Tablet, Cap or Capsule, Inj or Injection, Drop or Drops, Saline, in some cased there might have spelling mistakes like Taba or Cp with Tab or Cap or other misspells with Drops and Saline and correct the accordingly), Medicine Name (which is only start with an alphabet, but can not start with a number, do not include any nosily element as medicine name which may come from OCR missreading), Medicine power (such as *mg or **ml, ** means a number), Intake Frequency (where mention frequency like 1+1+1 or any other combination it may appears, but always check there should not be any abrupt number such as 8+1+0 or 5+0+1 or something appear like this, if you do not found anything just update it with "Not Found"), Frequency Days (which some times appears some time not, if appeare intake time will be indicated by ** inside a () this like bracket or ** with text "Days" or "Day", where ** must be a number, if you do not found anything just update it with "Not Found"). If there contain any unwanted or noisy data just remove it. For any case if you can not extract Medicine Type or Medicine Name for any entity then do not fill Medicine Dose, Intake Frequency, Frequency Days with gurbage value.

Here all ** means number.

Organize this data according to this format
Medicine Type:
Medicine Name:
Medicine Dose:
Intake Frequency:
Frequency Days:

"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [10]:
os.environ["WANDB_DISABLED"] = "false"

In [11]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [43]:
import pandas as pd
df = pd.read_csv("/content/ds.csv")
df.head()

,raw,formatted
0,Tab IDA 625mg (21)\r\n1+1+1\r\nTab lora 10mg (...,"{""medicines"": [{""type"": ""Tab"", ""name"": ""IDA 62..."
1,Tab IDA 625mg (21)\r\n1+1+1\r\nTab lora 10mg (...,"{""medicines"": [{""type"": ""Tab"", ""name"": ""IDA 62..."
2,mfelulena saline (1oooml)\r\nTaba saline (1ooo...,"{""medicines"": [\r\n {""type"": ""Saline"", ""name""..."
3,Tab IDA 625mg (21)\r\n1+1+1\r\nNapa 500mg(10)\...,"{""medicines"": [\r\n {""type"": ""Tab"", ""name"": ""..."
4,Tab Napa 500mg (20mg)\r\n2+2+2\r\nap ometid 20...,"{""medicines"": [\r\n {""type"": ""Tab"", ""name"": ""..."


In [44]:
from datasets import Dataset


train_dataset = Dataset.from_pandas(df)

data = {"train": train_dataset}
print(data["train"])


Dataset({
    features: ['raw', 'formatted'],
    num_rows: 15
})


In [45]:
data['train']['raw']

['Tab IDA 625mg (21)\r\n1+1+1\r\nTab lora 10mg (10)\r\nTab lora 10mg\r\n0+0+1\r\nCap- Ometid 20mg\r\n1+0+1\r\n8+0+1 2mg (10)\r\nTab Brizy 2mg(10)\r\nTab Brizy 2mg\r\n1+0+1\r\nTab Napa\xa0500mg\r\n2+2+2',
 'Tab IDA 625mg (21)\r\n1+1+1\r\nTab lora 10mg (10)\r\nTab lora 10mg\r\n0+0+1\r\nCap- Ometid 20mg\r\n1+0+1\r\n8+0+1 2mg (10)\r\nTab Brizy 2mg(10)\r\nTab Brizy 2mg\r\n1+0+1\r\nTab Napa 500mg\r\n2+2+2\r\nRhinozol nasal\r\ndrops 0.1% 2-3\r\ndrops in each\r\nnostril 4 times daily',
 "mfelulena saline (1oooml)\r\nTaba saline (1oooml)\r\nAab 20mg+2mg\r\nTab Neferine\r\nlike yy shall rely\r\nIwa %, Spiked ml\r\nTab Filmet\r\nmeal's 8mg\r\ncap Esotid\r\n2+0+0\r\nTab perieast\r\nOrs - 5",
 'Tab IDA 625mg (21)\r\n1+1+1\r\nNapa 500mg(10)\r\nTab Napa 500mg\r\n2+2+2 after meal\r\nCap Ometid 20mg\r\n1+0+1\r\nTab Brizy 2mg\r\n2mg\r\n1+0+1\r\n?200mg\r\nFixolin 200mg\r\nTab Fixalny 200\r\n1+0+1\r\nTab lora 10mg\r\n0+0+1\r\n0+0+1\r\n025l nasal drop.\r\nhrazol nasal nosol\r\nRhinozol nasal\r\n2-3 drops i

In [46]:
def formatting_func(example):
    text = f"Raw OCR: {example['raw'][0]}\nFormatted Medication: {example['formatted'][0]}"
    return [text]

In [51]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [52]:
trainer.train()

Step,Training Loss
10,0.313800
20,0.120500
30,0.016000
40,0.006500
50,0.002000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


TrainOutput(global_step=200, training_loss=0.02294669096110738, metrics={'train_runtime': 107.7276, 'train_samples_per_second': 7.426, 'train_steps_per_second': 1.857, 'total_flos': 726568599552000.0, 'train_loss': 0.02294669096110738, 'epoch': 200.0})

In [53]:
text = """Raw OCR: Tab IDA 625mg (21)
1+1+1
Tab lora 10mg (10)
Tab lora 10mg
0+0+1
Cap- Ometid 20mg
1+0+1
8+0+1 2mg (10)
Tab Brizy 2mg(10)
Tab Brizy 2mg
1+0+1
Tab Napa 500mg
2+2+2
Rhinozol nasal
drops 0.1% 2-3
drops in each
nostril 4 times daily"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Raw OCR: Tab IDA 625mg (21)
1+1+1
Tab lora 10mg (10)
Tab lora 10mg
0+0+1
Cap- Ometid 20mg
1+0+1
8+0+1 2mg (10)
Tab Brizy 2mg(10)
Tab Brizy 2mg
1+0+1
Tab Napa 500mg
2+2+2
Rhinozol nasal
drops 0.1% 2-3
drops in each
nostril 4 times daily
1+0+1
Tab N incontri 20mg
1+0+1
Tab Tab IDA 625mg
2+2+2
Tab Napa 500mg
1+0+1
Tab N Napa 500mg
2+2+2
Tab- Brizy 2mg
1+0+1
Tab Tab lora 10mg
2+2+2
Cap- Ometid 20mg
1+0+1
Tab Napa 500mg
2+2+2
Tab- Brizy 2mg
1+0+1
Tab Tab Napa 500mg
2+2+2
Tab- Napa 500mg
1+0+1
Tab Tab Brizy 2mg
2+2+2
Tab- N dorzol 2mg
1
